step - maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (30 days simulation).

type - CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER.

amount - amount of the transaction in local currency.

nameOrig - customer who started the transaction

oldbalanceOrg - initial balance before the transaction

newbalanceOrig - new balance after the transaction

nameDest - customer who is the recipient of the transaction

oldbalanceDest - initial balance recipient before the transaction. Note that there is not information for customers that start with M (Merchants).

newbalanceDest - new balance recipient after the transaction. Note that there is not information for customers that start with M (Merchants).

isFraud - This is the transactions made by the fraudulent agents inside the simulation. In this specific dataset the fraudulent behavior of the agents aims to profit by taking control or customers accounts and try to empty the funds by transferring to another account and then cashing out of the system.

isFlaggedFraud - The business model aims to control massive transfers from one account to another and flags illegal attempts. An illegal attempt in this dataset is an attempt to transfer more than 200.000 in a single transaction.

https://www.kaggle.com/arjunjoshua/predicting-fraud-in-financial-payment-services

frauddf['time_window'] =frauddf['step'].apply( lambda h: max(math.ceil((h-(math.floor(h/24))*24)/8),1))

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer
import pandas as pd

In [48]:

rawdata_df = pd.read_csv("PS_20174392719_1491204439457_log.csv")


#### EXPLORE RAW DATA

In [49]:
#Verifying the shape, rows and columns of the data frame
rawdata_df.shape

(6362620, 11)

In [50]:
#Verifying the column header
rawdata_df.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [51]:
#view the first five rows of the data frame to verify import
rawdata_df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


#### Feature Engineering

In [52]:

#Removing varaibles that we didn't think would contribute to the model
df_unused_col_removed = rawdata_df.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1)
df_unused_col_removed.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0


In [53]:

#Using One Hot Enoding to transfor our categorical variable Type to dummy variables
df_w_dummies = pd.get_dummies(df_unused_col_removed, columns=['type'])
df_w_dummies.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,170136.0,160296.36,0.0,0.0,0,0,0,0,1,0
1,1,1864.28,21249.0,19384.72,0.0,0.0,0,0,0,0,1,0
2,1,181.00,181.0,0.00,0.0,0.0,1,0,0,0,0,1
3,1,181.00,181.0,0.00,21182.0,0.0,1,0,1,0,0,0
4,1,11668.14,41554.0,29885.86,0.0,0.0,0,0,0,0,1,0


In [54]:
import math


df_w_dummies_and_step = df_w_dummies

df_w_dummies_and_step['time_window'] =df_w_dummies_and_step['step'].apply( lambda h: max(math.ceil((h-(math.floor(h/24))*24)/8),1))



In [55]:
df_w_dummies_and_step = df_w_dummies_and_step.drop(['step'], axis=1)
df_w_dummies_and_step.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,time_window
0,9839.64,170136.0,160296.36,0.0,0.0,0,0,0,0,1,0,1
1,1864.28,21249.0,19384.72,0.0,0.0,0,0,0,0,1,0,1
2,181.00,181.0,0.00,0.0,0.0,1,0,0,0,0,1,1
3,181.00,181.0,0.00,21182.0,0.0,1,0,1,0,0,0,1
4,11668.14,41554.0,29885.86,0.0,0.0,0,0,0,0,1,0,1


In [56]:
#performing MinMax scalar to all variables - we understand that this doesn't make sense for variable "step"

from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1), copy=True)
tmp_scaled_df = scaler.fit_transform(df_w_dummies_and_step)
scaled_df = pd.DataFrame(tmp_scaled_df)
scaled_df.head()
                                             

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000106,0.002855,0.003233,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.000020,0.000357,0.000391,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.000002,0.000003,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.000002,0.000003,0.000000,0.000059,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.000126,0.000697,0.000603,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [57]:
#SageMaker expects the y (output/predictor) to be in the first column slot therefore, moving that row 6 to first column of data frame

Y = scaled_df[5]

X = scaled_df.drop([5], axis=1)

finaldata = pd.concat([Y,X], axis=1)

finaldata.head()

,5,0,1,2,3,4,6,7,8,9,10,11
0,0.0,0.000106,0.002855,0.003233,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.000020,0.000357,0.000391,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.000002,0.000003,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.000002,0.000003,0.000000,0.000059,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.000126,0.000697,0.000603,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0


#### START THE MODEL STEP

In [58]:
import numpy as np

#Splitting data into train and validation and test (which will be used later as the hold out set)
train_data, validation_data, test_data = np.split(finaldata.sample(frac=1, random_state=1729), [int(0.7 * len(finaldata)), int(0.9 * len(finaldata))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

In [59]:
import boto3

#Defining variables
bucket = 'table1-aws-bankfraud'
prefix = 'sagemaker/DEMO-xgboost-churn'

#Putting csv files into s3
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

In [60]:
#Setting up the container
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost')

In [61]:
#Creating the train and validation channels
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

In [ ]:
#Setting up the XGBoost model and parameters to train the model with the train and validation data
sess = sagemaker.Session()
from sagemaker import get_execution_role
role = get_execution_role()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

2019-08-02 18:59:08 Starting - Starting the training job...
2019-08-02 18:59:10 Starting - Launching requested ML instances......
2019-08-02 19:00:15 Starting - Preparing the instances for training...
2019-08-02 19:01:05 Downloading - Downloading input data...
2019-08-02 19:01:35 Training - Downloading the training image...
2019-08-02 19:01:54 Training - Training image download completed. Training in progress.
Arguments: train
[2019-08-02:19:01:55:INFO] Running standalone xgboost training.
[2019-08-02:19:01:55:INFO] File size need to be processed in the node: 578.98mb. Available memory size in the node: 8433.92mb
[2019-08-02:19:01:55:INFO] Determined delimiter of CSV input is ','
[19:01:55] S3DistributionType set as FullyReplicated
[19:01:56] 4453834x11 matrix with 48992174 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2019-08-02:19:01:57:INFO] Determined delimiter of CSV input is ','
[19:01:57] S3DistributionType set as FullyReplicated
[19:01:57] 

In [ ]:
#Deploying the model to be used
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

In [ ]:
#Defining variables
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

In [ ]:
#Getting the endpoint name
xgb_predictor.endpoint

In [ ]:
test_data.head()
test_data_new = test_data.drop([5],axis=1)
test_data_new.head()

In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data_new.as_matrix())

In [ ]:
# pd.crosstab(index=test_data.iloc[:10000, 0], columns=np.round(predictions), rownames=['actual'], colnames=['predictions'])


pd.crosstab(np.round(test_data[5]), columns = np.where(predictions>0.3, 1, 0), rownames=['actual'], colnames=['predictions'])




In [ ]:
plt.hist(predictions)
plt.show()